In [1]:
import msoffcrypto
import io
import pandas as pd
from pyxlsb import open_workbook
import os
import numpy as np
import calendar

C:\Users\Neil\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
folder_path = "C:/Users/Neil/OneDrive/Documents/Capstone Files/Renamed_Datasets_Home"
password = "CRQAD_amf_2025!"

# List all .xlsb files in the folder
xlsb_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsb")]

dfs = {}  # Dictionary to store DataFrames

for file in xlsb_files:
    file_path = os.path.join(folder_path, file)

    decrypted = io.BytesIO()
    with open(file_path, "rb") as f:
        office_file = msoffcrypto.OfficeFile(f)
        office_file.load_key(password=password)
        office_file.decrypt(decrypted)

    # Read decrypted .xlsb file
    df = []
    with open_workbook(decrypted) as wb:
        with wb.get_sheet(1) as sheet:  # Read first sheet
            for row in sheet.rows():
                df.append([item.v for item in row])

    df = pd.DataFrame(df[1:], columns=df[0])  # Convert to pandas DataFrame
    dfs[file] = df  # Store DataFrame with file name as key

    print(f"Loaded: {file}")

Loaded: 01 ILS HOME JAN 2015 CONSO.xlsb
Loaded: 02 ILS HOME FEB 2015 CONSO.xlsb
Loaded: 03 ILS HOME MAR 2015 CONSO.xlsb
Loaded: 04 ILS HOME APR 2015 CONSO.xlsb
Loaded: 05 ILS HOME MAY 2015 CONSO.xlsb
Loaded: 06 ILS HOME JUN 2015 CONSO.xlsb
Loaded: 07 ILS HOME JUL 2015 CONSO.xlsb
Loaded: 08 ILS HOME AUG 2015 CONSO.xlsb
Loaded: 09 ILS HOME SEP 2015 CONSO.xlsb
Loaded: 10 ILS HOME OCT 2015 CONSO.xlsb
Loaded: 11 ILS HOME NOV 2015 CONSO.xlsb
Loaded: 12 ILS HOME DEC 2015 CONSO.xlsb
Loaded: 13 ILS HOME JAN 2016 CONSO.xlsb
Loaded: 14 ILS HOME FEB 2016 CONSO.xlsb
Loaded: 15 ILS HOME MAR 2016 CONSO.xlsb
Loaded: 16 ILS HOME APR 2016 CONSO.xlsb
Loaded: 17 ILS HOME MAY 2016 CONSO.xlsb
Loaded: 18 ILS HOME JUN 2016 CONSO.xlsb
Loaded: 19 ILS HOME JUL 2016 CONSO.xlsb
Loaded: 20 ILS HOME AUG 2016 CONSO.xlsb
Loaded: 21 ILS HOME SEP 2016 CONSO.xlsb
Loaded: 22 ILS HOME OCT 2016 CONSO.xlsb
Loaded: 23 ILS HOME NOV 2016 CONSO.xlsb
Loaded: 24 ILS HOME DEC 2016 CONSO.xlsb
Loaded: 25 ILS HOME JAN 2017 CONSO.xlsb


In [3]:
del_columns = ["DEL7", "DEL6", "DEL5", "DEL4", "DEL3", "DEL2", "DEL1"]

dfs_worst_delinquency = {}

for file, df in dfs.items():
    existing_del_cols = [col for col in del_columns if col in df.columns]

    # Compute worst delinquency level correctly
    worst_del = df[existing_del_cols].ne(0).values.argmax(axis=1)

    # Fix issue: If all values are zero, set worst_del to 0
    all_zero_mask = df[existing_del_cols].sum(axis=1) == 0  # Identify all-zero rows
    df["Worst_Del"] = 7 - worst_del
    df.loc[all_zero_mask, "Worst_Del"] = 0  # Assign 0 to rows with no delinquency

    dfs_worst_delinquency[file] = df

In [4]:
file_name = '01 ILS HOME JAN 2015 CONSO.xlsb'  # Replace with the actual file name
dfs_worst_delinquency[file_name][["DEL1", "DEL2", "DEL3", "DEL4", "DEL5", "DEL6", "DEL7", "Worst_Del"]].head(30)

,DEL1,DEL2,DEL3,DEL4,DEL5,DEL6,DEL7,Worst_Del
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,11.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,2
4,57.0,2.0,0.0,0.0,0.0,0.0,0.0,2
5,4.0,2.0,3.0,0.0,0.0,0.0,0.0,3
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
columns_to_rename = ["Worst_Del", "DEL1"]  # Include Worst_Del and DEL1

dfs_renamed = {}

for file, df in dfs_worst_delinquency.items():
    # Extract the month and year (always at the end before .xlsb)
    filename = file.replace(".xlsb", "")  # Remove extension
    parts = filename.split()  # Split by spaces
    month_year = " ".join(parts[-3:-1])  # Get last two parts as "JAN 2015"

    # Rename columns
    rename_dict = {col: f"{col} {month_year}" for col in columns_to_rename if col in df.columns}
    
    df = df.rename(columns=rename_dict)
    
    # Store modified DataFrame
    dfs_renamed[file] = df

# Example: Check a renamed DataFrame
print(dfs_renamed["01 ILS HOME JAN 2015 CONSO.xlsb"].head())

  APPRAISAL_DATE  APPRAISAL_RATE  APPRAISAL_VALUE   BOOK_DATE  BOOK_VALUE  \
0           None           100.0        4111714.0  11/28/2014  3996638.27   
1     10/05/2007           100.0        3480000.0  04/29/2008        0.00   
2     05/23/2011           100.0        1365200.0  05/23/2011   810996.05   
3     07/22/2010           100.0        4491000.0  02/24/2011  1751493.73   
4     02/05/2007           100.0        6000000.0  03/30/2007  1303672.02   

  CAR_DESCRIPTION CAR_MANUFACTURER CAR_MODEL  CLOSE_DATE  \
0            None             None      None        None   
1            None             None      None  03/18/2013   
2            None             None      None        None   
3            None             None      None        None   
4            None             None      None        None   

   COLLATERAL_CONDITION  ...  CLNT_TYPE  Total Interest Payment  \
0                   0.0  ...          C                47758.01   
1                   0.0  ...          C   

In [6]:
print(dfs_renamed["01 ILS HOME JAN 2015 CONSO.xlsb"].columns)

Index(['APPRAISAL_DATE', 'APPRAISAL_RATE', 'APPRAISAL_VALUE', 'BOOK_DATE',
       'BOOK_VALUE', 'CAR_DESCRIPTION', 'CAR_MANUFACTURER', 'CAR_MODEL',
       'CLOSE_DATE', 'COLLATERAL_CONDITION', 'COLLATERAL_TYPE',
       'CREDIT_RATING', 'PDO_MAT_DATE', 'DEL1 JAN 2015', 'DEL2', 'DEL3',
       'DEL4', 'DEL5', 'DEL6', 'DEL7', 'DELQ_AGE', 'DELQ_AGE_ACTUAL',
       'DOWNPAYMENT', 'EXPIRED', 'FACE_AMT', 'FINANCED_AMT', 'GL_STATUS',
       'GL_STATUS_DESCRIPTION', 'GROSS_MONTHLY_AMORTISATION', 'ILS_ACCT_NUM',
       'LAND_VALUE', 'LAST_PAID_AMT', 'LAST_PAID_DATE', 'LOAN_DUE', 'MATURED',
       'MATURITY', 'MATURITY_DUE_CHANGE', 'NMA', 'ORIG_BOOK_DATE',
       'ORIGINAL_RATE', 'OUTSTANDING_BAL', 'PREPAYMENT', 'PRODUCT_TYPE',
       'PROPERTY_TYPE', 'LOAN_TERM', 'WORST_DELINQUENCY', 'COLLATERAL_REFNUM',
       'PURPOSE_CODE', 'PURPOSE_DESC', 'REMEDIAL_STATUS_CODE',
       'PAST_DUE_STATUS', 'DEFAULT_TAGGING', 'EXCHANGE_RATE', 'SECURITY_TYPE',
       'RE_FLOOR_AREA', 'AVAILMENT', 'CAR_TYPE', 'LOA

In [7]:
key_column = "ILS_ACCT_NUM"

# List to store DataFrames with selected columns
dfs_lst = []

for file, df in dfs_renamed.items():
    # Identify columns that start with 'DEL1' and 'Worst_Del'
    del1_col = [col for col in df.columns if col.startswith("DEL1 ")]  # Includes month-year
    worst_del_col = [col for col in df.columns if col.startswith("Worst_Del ")]

    if key_column in df.columns and del1_col and worst_del_col:
        selected_cols = [key_column] + del1_col + worst_del_col
        dfs_lst.append(df[selected_cols])  # Select only relevant columns

# Merge all DataFrames using ILS_ACCT_NUM as the primary key
merged_df = dfs_lst[0]
for df in dfs_lst[1:]:
    merged_df = pd.merge(merged_df, df, on=key_column, how="outer")

# Save the merged result
merged_df.to_csv("merged_home_1.csv", index=False)

# Display the first 100 rows
print(merged_df.head(100))

       ILS_ACCT_NUM  DEL1 JAN 2015  Worst_Del JAN 2015  DEL1 FEB 2015  \
0   232001000330243            4.0                 1.0            4.0   
1   232001002826911            0.0                 0.0            0.0   
2   232001009874040            NaN                 NaN            NaN   
3   232002000129304            0.0                 0.0            0.0   
4   232002000651195           10.0                 1.0           10.0   
..              ...            ...                 ...            ...   
95  232009008221597            NaN                 NaN            NaN   
96  232009008836270            NaN                 NaN            NaN   
97  232009008838576            NaN                 NaN            NaN   
98  232009008915270            NaN                 NaN            NaN   
99  232009009184660            NaN                 NaN            NaN   

    Worst_Del FEB 2015  DEL1 MAR 2015  Worst_Del MAR 2015  DEL1 APR 2015  \
0                  1.0            4.0          

In [9]:
file_path = 'C:/Users/Neil/OneDrive/Documents/Capstone Files/Extracted Datasets/HOME/2024/12 ILS HOME DEC 2024 CONSO.xlsb'
password = "CRQAD_amf_2025!"

# Decrypt the file
decrypted = io.BytesIO()
with open(file_path, "rb") as f:
    office_file = msoffcrypto.OfficeFile(f)
    office_file.load_key(password=password)
    office_file.decrypt(decrypted)

# Read the decrypted `.xlsb` file
decrypted.seek(0)  # Reset pointer before reading

data = []
with open_workbook(decrypted) as wb:
    sheet_name = list(wb.sheets)[0]  # Get the first sheet name
    print(f"Opening Sheet: {sheet_name}")

    # Read sheet data
    with wb.get_sheet(sheet_name) as sheet:
        for row in sheet.rows():
            data.append([item.v for item in row])

# Convert to DataFrame
df = pd.DataFrame(data)
df.columns = df.iloc[0]  # Assign first row as column names
df = df[1:].reset_index(drop=True)  # Remove the first row from data

# Display first few rows
print(df.head())

Opening Sheet: Sheet1
0 APPRAISAL_DATE APPRAISAL_RATE APPRAISAL_VALUE   BOOK_DATE BOOK_VALUE  \
0           None          100.0             0.0  03/19/2003        0.0   
1           None           70.0       3192000.0  06/26/2008        0.0   
2           None           60.0        216000.0  10/25/2002        0.0   
3           None           70.0       1000000.0  05/23/2001        0.0   
4     09/19/2007          100.0        107100.0  12/10/2007        0.0   

0 CAR_DESCRIPTION CAR_MANUFACTURER CAR_MODEL  CLOSE_DATE COLLATERAL_CONDITION  \
0            None             None      None  03/18/2008                  0.0   
1            None             None      None  06/13/2012                  0.0   
2            None             None      None  10/25/2007                  0.0   
3            None             None      None  05/23/2011                  0.0   
4            None             None      None  08/09/2008                  0.0   

0  ...    RM_NO CLNT_TYPE Total Interest Payme

In [10]:
df

,APPRAISAL_DATE,APPRAISAL_RATE,APPRAISAL_VALUE,BOOK_DATE,BOOK_VALUE,CAR_DESCRIPTION,CAR_MANUFACTURER,CAR_MODEL,CLOSE_DATE,COLLATERAL_CONDITION,...,RM_NO,CLNT_TYPE,Total Interest Payment,Total Principal Payment,ADJ_DELQ_AGE_ACTUAL,PD,EDITED_PURPOSE,COLLATERAL_TYPE_DESCRIPTION,EDITED_COLLATERAL_TYPE,FOR REM
0,None,100.0,0.0,03/19/2003,0.0,None,None,None,03/18/2008,0.0,...,8972790,P,85110.58,434109.83,None,None,Others,REGISTERED REM,REM,Unspecified
1,None,70.0,3192000.0,06/26/2008,0.0,None,None,None,06/13/2012,0.0,...,2005978,P,103434.01,495538.53,None,None,Others,REGISTERED REM,REM,Unspecified
2,None,60.0,216000.0,10/25/2002,0.0,None,None,None,10/25/2007,0.0,...,94507,P,21866.43,99681.29,None,None,Others,REGISTERED REM,REM,Unspecified
3,None,70.0,1000000.0,05/23/2001,0.0,None,None,None,05/23/2011,0.0,...,266267,P,295423.71,774905.53,None,None,Others,REGISTERED REM,REM,Unspecified
4,09/19/2007,100.0,107100.0,12/10/2007,0.0,None,None,None,08/09/2008,0.0,...,8807804,P,26941.82,47446.76,None,None,Others,REGISTERED REM,REM,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55122,11/05/2024,100.0,7049167.55,12/05/2024,2184700.0,None,None,None,None,None,...,4912910341,P,0.0,0.0,0.0,0.0,Construction,REGISTERED REM,REM,House & Lot
55123,06/04/2024,100.0,50565943.0,12/16/2024,2000000.0,None,None,None,None,None,...,4912038039,P,0.0,0.0,0.0,0.0,PI-AP,REGISTERED REM,REM,Unspecified
55124,06/11/2024,100.0,8559500.0,12/13/2024,8525000.0,None,None,None,None,None,...,21093587,P,0.0,0.0,0.0,0.0,Acquisition,CLEAN,Others,Condominium
55125,11/25/2024,100.0,2334294.0,12/27/2024,3104395.9,None,None,None,None,None,...,9674814,P,0.0,0.0,0.0,0.0,Acquisition,CLEAN,Others,Condominium


In [11]:
key_column = "ILS_ACCT_NUM"

# Convert key column to string in both DataFrames
merged_df[key_column] = merged_df[key_column].astype(str)
df[key_column] = df[key_column].astype(str)

# Extract required columns
book_date_df = df[[key_column, "ORIG_BOOK_DATE", "MATURITY", "PRODUCT_TYPE"]]

# Merge, ensuring new columns come first
merged_df = pd.merge(book_date_df, merged_df, on=key_column, how="left")

print(merged_df.dtypes)  # Check data types to confirm
print(merged_df.head())  # Verify output

ILS_ACCT_NUM           object
ORIG_BOOK_DATE         object
MATURITY               object
PRODUCT_TYPE           object
DEL1 JAN 2015         float64
                       ...   
Worst_Del OCT 2024    float64
DEL1 NOV 2024         float64
Worst_Del NOV 2024    float64
DEL1 DEC 2024         float64
Worst_Del DEC 2024      int64
Length: 244, dtype: object
      ILS_ACCT_NUM ORIG_BOOK_DATE    MATURITY PRODUCT_TYPE  DEL1 JAN 2015  \
0  266517000260174     03/19/2003  03/18/2008        177.0            3.0   
1  266070000201309     06/13/2002  06/13/2012        177.0            0.0   
2  261857000273764     10/25/2002  10/25/2007        174.0            2.0   
3  261218000267639     05/23/2001  05/23/2011        174.0           16.0   
4  261220000155926     12/10/2007  12/10/2012        174.0            0.0   

   Worst_Del JAN 2015  DEL1 FEB 2015  Worst_Del FEB 2015  DEL1 MAR 2015  \
0                 1.0            3.0                 1.0            3.0   
1                 0.0         

In [12]:
merged_df.to_csv("ILS_home_final.csv", index=False)